# ASTR 420 - Week 6:  "Family Traits of Nebulae" 

The Hubble system of galaxy morphological classification is illustrated in the well-known tuning-fork diagram, published in his 1936 book "The Realm of the Nebulae".  This has often been misinterpreted as a diagram for galaxy evolution, but it was really only ever intended for classification purposes. As we will find out later, galaxy evolution is a much more complicated process than what might be naively assumed from the historical terminology of "early-type" and "late-type" galaxies. 

![TuningFork](img/TuningFork.jpeg)

With the advent of the vast Sloan Digital Sky Survey, which has imaged one quarter of the night sky, we now have detailed images of hundreds of thousands of galaxies in the local universe.  The SDSS DR8 also includes data on visual galaxy morphologies from the 
 [Galaxy Zoo](https://www.zooniverse.org/projects/zookeeper/galaxy-zoo/) project. Galaxy Zoo is a web-based project
 that used the collective efforts of about 100,000 volunteer 
 citizen scientists to classify more than one million
 SDSS DR7 Main Galaxy Sample (MGS) galaxies.
 Full details on the classification process, including the operation
 of the site and estimates of classification accurary, are given in
 [Lintott et al. (2008)](http://adsabs.harvard.edu/abs/2008MNRAS.389.1179L).
 
 Today we'll use SDSS data and GalaxyZoo morphological classifications to explore how various properties of galaxies like color, shape, and magnitude are related. 
 
 Let's start, as always, by importing all of the most important libraries that we're likely to need...

In [1]:
# Import Python libraries to work with SciServer (see http://www.sciserver.org/docs/sciscript-python/)

import SciServer.CasJobs as CasJobs # query with CasJobs, the primary database for the SDSS
import SciServer.SkyServer as SkyServer # show individual objects through SkyServer
import SciServer.SciDrive           # read/write to/from SciDrive, cloud-based storage for your work
import numpy as np                  # standard Python library for math operations
import pandas                       # package for structuring and visualizing data
import matplotlib.pyplot as plt     # package for graphing and visualization 
print('SciServer libraries imported')

# Apply some special settings to the imported libraries

# ensures columns get written completely in notebook
pandas.set_option('display.max_colwidth', -1)
# do *not* show python warnings 
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Image, SVG, Math
print('Settings applied')

# So that we're able to make plots, import matplotlib and some of its modules

import matplotlib
%config InlineBackend.rc = {}
from matplotlib import pylab as plt
%matplotlib inline
print('OK')


SciServer libraries imported
Settings applied
OK


 ------------------------
 ### Exploring Morphological Trends with GalaxyZoo

Galaxy Zoo data are contained in tables that begin with "zoo"
in the [SDSS Table schema](http://skyserver.sdss.org/dr15/en/help/browser/browser.aspx#&&history=shortdescr+Tables+U). For each galaxy, available data includes its
SDSS object ID and weighted probabilities for each of six different
morphological classifications based on counts of volunteer "votes".
Full details on what data the Galaxy Zoo catalog contains are given
in [Lintott et al. (2010)](http://adsabs.harvard.edu/abs/2011MNRAS.410..166L).

The example query below finds the weighted probability that a given galaxy has each of the following six morphological classifications:

elliptical: 'p_el'

spiral (clockwise winding):  'p_cw' 

spiral (counter-clockwise winding): 'p_acw' 

edge-on: 'p_edge'

merger: 'p_mg'

undetermined: 'p_dk'


In [2]:
# Here's an example query to return 100 galaxies that have spectra, clean photometry,
# at least 10 Galaxy Zoo volunteer votes, and at least an 80% probability
# of being clockwise spirals.

query="""
SELECT TOP 100 p.objid, p.ra,p.dec,p.petror90_r, p.dered_g, s.z as z_spec, s.zErr, zns.nvote, zns.p_el as elliptical, zns.p_cw as spiralclock, zns.p_acw as spiralanticlock, zns.p_edge as edgeon, zns.p_dk as dontknow, zns.p_mg as merger
  FROM Galaxy as p
   JOIN ZooNoSpec AS zns ON p.objid = zns.objid
   JOIN SpecObj AS s ON s.bestobjid = p.objid
WHERE p.clean=1 and zns.nvote >= 10 and zns.p_cw > 0.8
"""
zoo_gals = CasJobs.executeQuery(query, "dr15")
zoo_gals = zoo_gals.set_index('objid')
zoo_gals

,ra,dec,petror90_r,dered_g,z_spec,zErr,nvote,elliptical,spiralclock,spiralanticlock,edgeon,dontknow,merger
objid,,,,,,,,,,,,,
1237668650926342749,241.829163,-0.443972,9.176703,17.18769,0.074706,0.000012,20,0.000,0.850,0.050,0.100,0.000,0.000
1237668650926539340,242.274702,-0.574642,7.471252,16.53492,0.056719,0.000008,32,0.031,0.906,0.000,0.062,0.000,0.000
1237648673458684355,244.496791,-0.498398,9.355639,16.13657,0.050477,0.000009,23,0.000,1.000,0.000,0.000,0.000,0.000
1237655560403354055,248.120390,-0.478861,7.999845,16.81796,0.070390,0.000023,40,0.075,0.800,0.000,0.100,0.000,0.025
1237648704064454738,238.736718,-0.209378,11.107280,15.60193,0.042800,0.000007,37,0.000,0.946,0.054,0.000,0.000,0.000
1237648673995686549,244.875590,-0.038526,12.179800,16.53927,0.079218,0.000027,35,0.000,0.971,0.029,0.000,0.000,0.000
1237655550206804790,247.424991,-0.167082,8.663761,17.34978,0.044132,0.000015,52,0.038,0.808,0.019,0.058,0.019,0.058
1237655554497315264,247.530807,-0.193460,9.905840,15.91343,0.046768,0.000025,31,0.032,0.935,0.000,0.000,0.000,0.032
1237668569320980980,240.498466,0.319765,9.797406,17.35759,0.041010,0.000021,22,0.000,0.818,0.000,0.182,0.000,0.000


#### **Your turn:  In the cell below, write a search of the [zoo2MainSpecz](https://skyserver.sdss.org/dr12/en/help/browser/browser.aspx#&&history=description+zoo2MainSpecz+U) Table to retrieve the major types of galaxies in the Hubble's "Tuning Fork" diagram.  Measure the average color for a large sample of each.**

A description of the method used to collect the data in the GalaxyZoo 2 data table are described in [Willet et al. 2013](http://adsabs.harvard.edu/cgi-bin/nph-data_query?bibcode=2013MNRAS.435.2835W&link_type=ARTICLE&db_key=AST&high=).  A flowchart of the questions used to determine morphological characteristics is shown below.
![GalaxyZoo2](img/Willet1.png)



----------------------
## Adding morphological information to the Galaxy Color-Magnitude Diagram

Earlier in this course you made a version of the [galaxy color-magnitude diagram](https://en.wikipedia.org/wiki/Galaxy_color%E2%80%93magnitude_diagram) (e.g., g vs. g-r).  By fetching a sample of galaxies with high signal-to-noise (e.g., g/gErr > 10) and a small redshift range (e.g., 0.1<z<0.12), you should have been able to resolve what is known as the "red sequence" and the "blue cloud".  GalaxyZoo provides us with the information to look for morphological trends in that diagram.

#### By querying the GalaxyZoo tables, explore where on the color-magnitude diagram galaxies of different Hubble "types" are preferentially found.
(This time I haven't given you a piece of code to start with, so you might find it helpful to go grab some working code from other notebooks you've completed...!)

#### Comment on your findings here.

-------------
## Exploring the "Green Valley"

Try displaying some cutout images of galaxies that fall in between the "red sequence" and the "blue cloud".  Is there any trend in their morphologies?


#### Comment on your findings here.


------------------------
## Exploring potential bias in GalaxyZoo data, Part 1

GalaxyZoo works because people are naturally amazing at pattern recognition, even for difficult shapes like spirals, which have historically challenged artificial intelligence.  But our ability to quickly detect patterns can also mean that we are suceptable to bias.

Let's try a couple of quick tests:

First, what fraction of the spirals are identified in GalaxyZoo as "clockwise", compared to "counter-clockwise"?



#### Comment on your findings here.

Note: GalaxyZoo scientists have done extensive testing on this question about how human pattern recognition may be better or worse when observing spirals that wind in a particular direction.   The data for results from their tests can be quereied from the [zooMirrorBias](http://skyserver.sdss.org/dr15/en/help/browser/browser.aspx#&&history=description+zooMirrorBias+U) table.

------------------------
## Exploring potential bias in GalaxyZoo data, Part 2

Next, does the color distribution of "spirals" and "ellipticals" appear any different if the results are obtained by only showing users monochromatic (i.e., black and white) images?  

Explore this question by querying the SDSS table [zooMonochromeBias](https://skyserver.sdss.org/dr12/en/help/browser/browser.aspx#&&history=description+zooMonochromeBias+U), which contains GalaxyZoo results from monochrome images. 

#### Comment on your findings here.